In [379]:
#import cx_Oracle
import pandas as pd
import sqlalchemy
from sqlalchemy.exc import SQLAlchemyError

<h1>Pobieranie danych z bazy</h1>

In [380]:
data = {}

dbtables = ['ONI_CIRCUITS', 'MEB_CONTAINERS', 'MEB_DGM', 'MEB_DMC', 'MEB_DMC_GRADE', 'MEB_GROB', 'MEB_KO', 'MEB_KO_DGM', 'MEB_KO_RODZAJ', 'MEB_KO_STREFA', 'MEB_KS', 'MEB_KS_WZORC']
username = 'dmcro'
password = 'ouZ9Rys7astDp4byjUvO'
dsn = 'vwpnwrud0000rac:1521/dmc.vwg'
dbhostname='vwpnwrud0000rac'
service_name='dmc.vwg'

In [381]:
try:
    sqlalchemy_engine="oracle+cx_oracle://"+username+":"+password+"@"+dbhostname+"/?service_name="+service_name
    engine = sqlalchemy.create_engine(sqlalchemy_engine, arraysize=1000)
    for table in dbtables:
        query = f"select * from z3dmc.{table} where rownum<101"
        data.update({table: pd.read_sql(query, engine)})
except SQLAlchemyError as e:
    print(e)

<h1>Łączenie tabel</h1>

In [382]:
# łączę tabele MEB_KO i MEB_KO_DGM z tabelami MEB_KO_STREFA/RODZAJ

data['MEB_KO'] = data['MEB_KO'].merge(data['MEB_KO_STREFA'], left_on='nok_strefa', right_on='indeks', how='inner')
data['MEB_KO'] = data['MEB_KO'].merge(data['MEB_KO_RODZAJ'], left_on='nok_rodzaj', right_on='indeks', how='inner')

data['MEB_KO_DGM'] = data['MEB_KO_DGM'].merge(data['MEB_KO_STREFA'], left_on='nok_strefa', right_on='indeks', how='inner')
data['MEB_KO_DGM'] = data['MEB_KO_DGM'].merge(data['MEB_KO_STREFA'], left_on='nok_rodzaj', right_on='indeks', how='inner')

# tu do wyrzucenia będzie kolumna indeks
#data['MEB_KO'].head()
# tu do wyrzucenia będzie kolumna indeks i kod_pola
#data['MEB_KO_DGM'].head()

In [383]:
# łączę tabelę MEB_DMC z tabelą MEB_KO

data['MEB_DMC'] = data['MEB_DMC'].merge(data['MEB_KO'], on='id_dmc', how='left')

#  do wyrzucenia kolumna timestamp oraz eks

data['MEB_DMC'].head()

,id_dmc,dmc,status_koncowy,timestamp_x,update_time,id_meb_containers,packed_time,first_packed_time,dmc_casting,production_step,...,nok_strefa,nok_rodzaj,status_ko,data,timestamp_y,eks,kod_pola,indeks_x,rodzaj_uszkodzenia,indeks_y
0,2063,0MH301103CVW41910090436321102S111910040455,2,2019-10-09 08:09:07,2019-11-06 05:40:27,366,2019-11-06 05:40:26,None,None,700.0,...,0.0,0.0,1.0,2019-10-09 08:19:50,2019-10-09 08:19:51,None,Brak,0.0,Brak,0.0
1,2064,0MH301103CVW41910090436411102S111910040410,1,2019-10-09 08:09:27,2019-11-06 05:50:29,0,2019-10-09 08:18:06,None,None,500.0,...,0.0,0.0,1.0,2019-10-09 08:17:07,2019-10-09 08:17:08,None,Brak,0.0,Brak,0.0
2,2065,0MH301103CVW41910090436521102S111910040350,2,2019-10-09 08:13:58,2019-11-06 06:47:19,366,2019-11-06 06:47:18,None,None,700.0,...,0.0,0.0,1.0,2019-10-09 08:16:03,2019-10-09 08:16:04,None,Brak,0.0,Brak,0.0
3,2066,0MH301103CVW41910090436611102S111910040356,2,2019-10-09 08:17:45,2019-11-06 05:40:39,366,2019-11-06 05:40:38,None,None,700.0,...,0.0,0.0,1.0,2019-10-09 08:39:11,2019-10-09 08:39:11,None,Brak,0.0,Brak,0.0
4,2067,0MH301103CVW41910090436721102S111910040345,2,2019-10-09 08:18:05,2019-10-09 08:22:31,0,NaT,None,None,NaN,...,513.0,202.0,2.0,2019-10-09 08:22:31,2019-10-09 08:22:31,None,S13,513.0,Mechaniczne - obróbka MF,202.0


In [384]:
# łączę tabelę MEB_DGM z tabelą MEB_KO_DGM

data['MEB_DGM'] = data['MEB_DGM'].merge(data['MEB_KO_DGM'], left_on='id', right_on='id_dmc', how='left')
data['MEB_DGM'].head()

,id,dmc,nr_dgm,status,czas_fazy_1,czas_fazy_2,czas_fazy_3,max_predkosc,cisnienie_tloka,cisnienie_koncowe,...,data_odlania_y,timestamp_y,operator,rodzaj_kontroli,product_id,line_id,kod_pola_x,indeks_x,kod_pola_y,indeks_y
0,220,190627091611011003610,10,4,1850,88,47.5,6.52,7,264,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3416,190719071611011006077,10,1,2197,89,46.5,6.59,11,285,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3417,190719071811011006078,10,1,2184,86,45.0,6.52,8,284,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3418,190719072011011006079,10,1,2159,89,45.5,6.54,6,284,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3419,190719072111011006080,10,1,2175,89,47.0,6.57,8,283,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [385]:
# przygotowywuję tabelę ONI_CIRCUITS do połączenia 

oni_circuits = data['ONI_CIRCUITS'].pivot(index='id_dmc', columns='circuit_nr', values=['assigment', 'flow', 'ref_time', 'set_point', 'start_delay', 'temp', 'working_mode'])
oni_circuits.columns = oni_circuits.columns.map('{0[0]}_{0[1]}'.format) 
oni_circuits.reset_index(inplace=True)


# łączę z tabelą MEB_DGM

oni_circuits = oni_circuits.merge(data['MEB_DGM'], on='id_dmc', how='left')
oni_circuits.head()

,id_dmc,assigment_1,assigment_2,assigment_3,assigment_4,assigment_6,assigment_7,assigment_8,assigment_9,assigment_10,...,data_odlania_y,timestamp_y,operator,rodzaj_kontroli,product_id,line_id,kod_pola_x,indeks_x,kod_pola_y,indeks_y
0,319248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,319320,NaN,NaN,NaN,NaN,NaN,NaN,17.0,NaN,NaN,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,319339,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,319340,NaN,NaN,NaN,NaN,NaN,17.0,NaN,NaN,NaN,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,319399,NaN,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [386]:
# łączę tabelę MEB_DMC z ONI_CIRCUITS

data['MEB_DMC'] = data['MEB_DMC'].merge(oni_circuits, on='id_dmc', how='left')
data['MEB_DMC'].head()

,id_dmc,dmc_x,status_koncowy,timestamp_x_x,update_time,id_meb_containers,packed_time,first_packed_time,dmc_casting,production_step,...,data_odlania_y,timestamp_y_y,operator,rodzaj_kontroli,product_id,line_id,kod_pola_x,indeks_x_y,kod_pola_y,indeks_y_y
0,2063,0MH301103CVW41910090436321102S111910040455,2,2019-10-09 08:09:07,2019-11-06 05:40:27,366,2019-11-06 05:40:26,None,None,700.0,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2064,0MH301103CVW41910090436411102S111910040410,1,2019-10-09 08:09:27,2019-11-06 05:50:29,0,2019-10-09 08:18:06,None,None,500.0,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2065,0MH301103CVW41910090436521102S111910040350,2,2019-10-09 08:13:58,2019-11-06 06:47:19,366,2019-11-06 06:47:18,None,None,700.0,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2066,0MH301103CVW41910090436611102S111910040356,2,2019-10-09 08:17:45,2019-11-06 05:40:39,366,2019-11-06 05:40:38,None,None,700.0,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2067,0MH301103CVW41910090436721102S111910040345,2,2019-10-09 08:18:05,2019-10-09 08:22:31,0,NaT,None,None,NaN,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [387]:
# łączę tabelę MEB_DMC z tabelą MEB_GROB

data['MEB_DMC'] = data['MEB_DMC'].merge(data['MEB_GROB'], on='id_dmc', how='left')
data['MEB_DMC'].head()

,id_dmc,dmc_x,status_koncowy,timestamp_x_x,update_time,id_meb_containers,packed_time,first_packed_time,dmc_casting,production_step,...,workingstep1,workingstep2,workingstep3,workingstep4,mms_ok,reworkrequested,reworkdone,partcleaningisfinished,waitfortoolcheck,machine_nr
0,2063,0MH301103CVW41910090436321102S111910040455,2,2019-10-09 08:09:07,2019-11-06 05:40:27,366,2019-11-06 05:40:26,None,None,700.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2064,0MH301103CVW41910090436411102S111910040410,1,2019-10-09 08:09:27,2019-11-06 05:50:29,0,2019-10-09 08:18:06,None,None,500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2065,0MH301103CVW41910090436521102S111910040350,2,2019-10-09 08:13:58,2019-11-06 06:47:19,366,2019-11-06 06:47:18,None,None,700.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2066,0MH301103CVW41910090436611102S111910040356,2,2019-10-09 08:17:45,2019-11-06 05:40:39,366,2019-11-06 05:40:38,None,None,700.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2067,0MH301103CVW41910090436721102S111910040345,2,2019-10-09 08:18:05,2019-10-09 08:22:31,0,NaT,None,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [388]:
# łączę tabelę MEB_DMC z tabelą MEB_KS

data['MEB_DMC'] = data['MEB_DMC'].merge(data['MEB_KS'], on='id_dmc', how='left')
data['MEB_DMC'].head()

,id_dmc,dmc_x,status_koncowy,timestamp_x_x,update_time,id_meb_containers,packed_time,first_packed_time,dmc_casting,production_step,...,przeciek,nrprogramu,temperaturatestu,statusszczelnosc,gradedmc_max,gradedmc_aktualny,statusdmc,nrformy,data_y,timestamp_y
0,2063,0MH301103CVW41910090436321102S111910040455,2,2019-10-09 08:09:07,2019-11-06 05:40:27,366,2019-11-06 05:40:26,None,None,700.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
1,2064,0MH301103CVW41910090436411102S111910040410,1,2019-10-09 08:09:27,2019-11-06 05:50:29,0,2019-10-09 08:18:06,None,None,500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
2,2065,0MH301103CVW41910090436521102S111910040350,2,2019-10-09 08:13:58,2019-11-06 06:47:19,366,2019-11-06 06:47:18,None,None,700.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
3,2066,0MH301103CVW41910090436611102S111910040356,2,2019-10-09 08:17:45,2019-11-06 05:40:39,366,2019-11-06 05:40:38,None,None,700.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
4,2067,0MH301103CVW41910090436721102S111910040345,2,2019-10-09 08:18:05,2019-10-09 08:22:31,0,NaT,None,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT


In [389]:
"""connection=cx_Oracle.connect(username,password,dsn,encoding='UTF-8')
connection.version
connection.close()"""

"connection=cx_Oracle.connect(username,password,dsn,encoding='UTF-8')\nconnection.version\nconnection.close()"

In [390]:
"""try:
    with cx_Oracle.connect(username,password,dsn,encoding='UTF-8') as connection:
        with connection.cursor() as cursor:
            cursor.execute(dmc_sql)
            while True:
                row = cursor.fetchone()
                if row is None:
                    break
                print(row)
                data={'id':row[0],'dmc':row[1]}
    cursor.close()
    connection.close()
except cx_Oracle.Error as error:
    print(error)"""

"try:\n    with cx_Oracle.connect(username,password,dsn,encoding='UTF-8') as connection:\n        with connection.cursor() as cursor:\n            cursor.execute(dmc_sql)\n            while True:\n                row = cursor.fetchone()\n                if row is None:\n                    break\n                print(row)\n                data={'id':row[0],'dmc':row[1]}\n    cursor.close()\n    connection.close()\nexcept cx_Oracle.Error as error:\n    print(error)"

In [391]:
#dodej to Adam przystojniaku